# EOD data inspection

In [13]:
import seaborn as sns
import pandas as pd
import geopandas as gpd

%matplotlib inline
sns.set(context='notebook', font='Lucida Sans Unicode', style='white', palette='plasma')

Import EOD(Encuesta Origen Destino) data

In [14]:
eod_trips = pd.read_csv('../data/other_format/EOD_STGO/viajes.csv', sep=';')
eod_trips = eod_trips[['Persona', 'Viaje', 'OrigenCoordX', 'OrigenCoordY', 'DestinoCoordX', 'DestinoCoordY', 'HoraIni', 'HoraFin']]
eod_trips

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin
0,17343102,1734310202,"335208,7188",6288387,"338812,3125",6292391,22:30,23:40
1,17344101,1734410101,"338536,4375",6291928,"354267,3438",6302297,13:00,14:45
2,17344101,1734410102,"354267,3438",6302297,"338536,4375",6291928,22:00,23:30
3,17344103,1734410301,"338536,4375",6291928,"350841,6563",6297212,9:00,9:55
4,17344103,1734410302,"350841,6563",6297212,"338536,4375",6291928,19:00,21:30
...,...,...,...,...,...,...,...,...
113586,74381301,7438130102,"349556,9063","6290333,5","349410,2813",6289669,12:00,12:15
113587,74381302,7438130201,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:35
113588,74381302,7438130202,"349556,9063","6290333,5","349410,2813",6289669,12:00,12:15
113589,74382301,7438230101,NaN,NaN,NaN,NaN,7:30,7:40


Delete NaN values in trips

In [21]:
trips_cleaned = eod_trips.dropna()
trips_cleaned

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin
0,17343102,1734310202,"335208,7188",6288387,"338812,3125",6292391,22:30,23:40
1,17344101,1734410101,"338536,4375",6291928,"354267,3438",6302297,13:00,14:45
2,17344101,1734410102,"354267,3438",6302297,"338536,4375",6291928,22:00,23:30
3,17344103,1734410301,"338536,4375",6291928,"350841,6563",6297212,9:00,9:55
4,17344103,1734410302,"350841,6563",6297212,"338536,4375",6291928,19:00,21:30
...,...,...,...,...,...,...,...,...
113584,70679101,7067910102,"338450,4063","6299725,5","338536,5625","6299639,5",12:40,12:48
113585,74381301,7438130101,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:45
113586,74381301,7438130102,"349556,9063","6290333,5","349410,2813",6289669,12:00,12:15
113587,74381302,7438130201,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:35


Functions to help in creating datetime object from the time data column (H:m), this will also help later on when sorting the trips, correcting the arrival time if necessary.

In [22]:
from datetime import datetime, timedelta

def set_origin_time(hour_origin):
    dt1 = datetime.strptime(hour_origin, '%H:%M').replace(year=2012, month=1, day=1) # Set dummy date(year, month,day)
    return dt1

def set_destination_time(row):
    hour_origin = row['HoraIni']
    hour_destination = row['HoraFin']
    dt1 = datetime.strptime(hour_origin, '%H:%M').replace(year=2012, month=1, day=1) # Set dummy date(year, month,day)
    dt2 = datetime.strptime(hour_destination, '%H:%M').replace(year=2012, month=1, day=1) # Set dummy date(year, month,day)
    time_difference = dt2 - dt1
    if time_difference.total_seconds() < 0:
        return dt2 + timedelta(days=1)
    else:
        return dt2

In [23]:
pd.options.mode.chained_assignment = None
trips_cleaned['o_time'] = trips_cleaned['HoraIni'].apply(set_origin_time)
trips_cleaned['d_time'] = trips_cleaned.apply(set_destination_time, axis=1)
trips_cleaned 

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin,o_time,d_time
0,17343102,1734310202,"335208,7188",6288387,"338812,3125",6292391,22:30,23:40,2012-01-01 22:30:00,2012-01-01 23:40:00
1,17344101,1734410101,"338536,4375",6291928,"354267,3438",6302297,13:00,14:45,2012-01-01 13:00:00,2012-01-01 14:45:00
2,17344101,1734410102,"354267,3438",6302297,"338536,4375",6291928,22:00,23:30,2012-01-01 22:00:00,2012-01-01 23:30:00
3,17344103,1734410301,"338536,4375",6291928,"350841,6563",6297212,9:00,9:55,2012-01-01 09:00:00,2012-01-01 09:55:00
4,17344103,1734410302,"350841,6563",6297212,"338536,4375",6291928,19:00,21:30,2012-01-01 19:00:00,2012-01-01 21:30:00
...,...,...,...,...,...,...,...,...,...,...
113584,70679101,7067910102,"338450,4063","6299725,5","338536,5625","6299639,5",12:40,12:48,2012-01-01 12:40:00,2012-01-01 12:48:00
113585,74381301,7438130101,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:45,2012-01-01 10:30:00,2012-01-01 10:45:00
113586,74381301,7438130102,"349556,9063","6290333,5","349410,2813",6289669,12:00,12:15,2012-01-01 12:00:00,2012-01-01 12:15:00
113587,74381302,7438130201,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:35,2012-01-01 10:30:00,2012-01-01 10:35:00


Delete trips with null origin or destination

In [24]:
trips_cleaned = (trips_cleaned
                 .pipe(lambda x: x[x.OrigenCoordX != '0'])
                 .pipe(lambda x: x[x.DestinoCoordX != '0'])
                 .pipe(lambda x: x[x.OrigenCoordY != '0'])
                 .pipe(lambda x: x[x.DestinoCoordY != '0'])
                 )
trips_cleaned

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin,o_time,d_time
0,17343102,1734310202,"335208,7188",6288387,"338812,3125",6292391,22:30,23:40,2012-01-01 22:30:00,2012-01-01 23:40:00
1,17344101,1734410101,"338536,4375",6291928,"354267,3438",6302297,13:00,14:45,2012-01-01 13:00:00,2012-01-01 14:45:00
2,17344101,1734410102,"354267,3438",6302297,"338536,4375",6291928,22:00,23:30,2012-01-01 22:00:00,2012-01-01 23:30:00
3,17344103,1734410301,"338536,4375",6291928,"350841,6563",6297212,9:00,9:55,2012-01-01 09:00:00,2012-01-01 09:55:00
4,17344103,1734410302,"350841,6563",6297212,"338536,4375",6291928,19:00,21:30,2012-01-01 19:00:00,2012-01-01 21:30:00
...,...,...,...,...,...,...,...,...,...,...
113584,70679101,7067910102,"338450,4063","6299725,5","338536,5625","6299639,5",12:40,12:48,2012-01-01 12:40:00,2012-01-01 12:48:00
113585,74381301,7438130101,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:45,2012-01-01 10:30:00,2012-01-01 10:45:00
113586,74381301,7438130102,"349556,9063","6290333,5","349410,2813",6289669,12:00,12:15,2012-01-01 12:00:00,2012-01-01 12:15:00
113587,74381302,7438130201,"349410,2813",6289669,"349556,9063","6290333,5",10:30,10:35,2012-01-01 10:30:00,2012-01-01 10:35:00


Fix the coord format and the geographic coordinate system code, also the h3 cells are uncluded

In [25]:
trips_cleaned['OrigenCoordX'] = trips_cleaned['OrigenCoordX'].apply(lambda x: x.replace(",", "."))
trips_cleaned['OrigenCoordY'] = trips_cleaned['OrigenCoordY'].apply(lambda x: x.replace(",", "."))
trips_cleaned['DestinoCoordX'] = trips_cleaned['DestinoCoordX'].apply(lambda x: x.replace(",", "."))
trips_cleaned['DestinoCoordY'] = trips_cleaned['DestinoCoordY'].apply(lambda x: x.replace(",", "."))

In [26]:
import h3
resolution = 12

current_crs = 'EPSG:5361'
target_crs = 'EPSG:4326'

origin_points = gpd.points_from_xy(trips_cleaned.OrigenCoordX, trips_cleaned.OrigenCoordY, crs=current_crs).to_crs(target_crs)
dest_points = gpd.points_from_xy(trips_cleaned.DestinoCoordX, trips_cleaned.DestinoCoordY, crs=current_crs).to_crs(target_crs)

trips_cleaned['o_lat'] = [p.y for p in origin_points]
trips_cleaned['o_lon'] = [p.x for p in origin_points]
trips_cleaned['d_lat'] = [p.y for p in dest_points]
trips_cleaned['d_lon'] = [p.x for p in dest_points]
trips_cleaned['o_h3_cell'] = [h3.latlng_to_cell(p.y, p.x, resolution) for p in origin_points]
trips_cleaned['d_h3_cell'] = [h3.latlng_to_cell(p.y, p.x, resolution) for p in dest_points]
trips_cleaned

,Persona,Viaje,OrigenCoordX,OrigenCoordY,DestinoCoordX,DestinoCoordY,HoraIni,HoraFin,o_time,d_time,o_lat,o_lon,d_lat,d_lon,o_h3_cell,d_h3_cell
0,17343102,1734310202,335208.7188,6288387,338812.3125,6292391,22:30,23:40,2012-01-01 22:30:00,2012-01-01 23:40:00,-33.531422,-70.774666,-33.495874,-70.735153,8cb2c540d0a89ff,8cb2c5429b60dff
1,17344101,1734410101,338536.4375,6291928,354267.3438,6302297,13:00,14:45,2012-01-01 13:00:00,2012-01-01 14:45:00,-33.500007,-70.738205,-33.408776,-70.567232,8cb2c5429ae5dff,8cb2c519a994dff
2,17344101,1734410102,354267.3438,6302297,338536.4375,6291928,22:00,23:30,2012-01-01 22:00:00,2012-01-01 23:30:00,-33.408776,-70.567232,-33.500007,-70.738205,8cb2c519a994dff,8cb2c5429ae5dff
3,17344103,1734410301,338536.4375,6291928,350841.6563,6297212,9:00,9:55,2012-01-01 09:00:00,2012-01-01 09:55:00,-33.500007,-70.738205,-33.454153,-70.604904,8cb2c5429ae5dff,8cb2c554db113ff
4,17344103,1734410302,350841.6563,6297212,338536.4375,6291928,19:00,21:30,2012-01-01 19:00:00,2012-01-01 21:30:00,-33.454153,-70.604904,-33.500007,-70.738205,8cb2c554db113ff,8cb2c5429ae5dff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113584,70679101,7067910102,338450.4063,6299725.5,338536.5625,6299639.5,12:40,12:48,2012-01-01 12:40:00,2012-01-01 12:48:00,-33.429695,-70.737727,-33.430483,-70.736816,8cb2c55509217ff,8cb2c55509297ff
113585,74381301,7438130101,349410.2813,6289669,349556.9063,6290333.5,10:30,10:45,2012-01-01 10:30:00,2012-01-01 10:45:00,-33.521961,-70.621567,-33.515990,-70.619877,8cb2c50927283ff,8cb2c50921181ff
113586,74381301,7438130102,349556.9063,6290333.5,349410.2813,6289669,12:00,12:15,2012-01-01 12:00:00,2012-01-01 12:15:00,-33.515990,-70.619877,-33.521961,-70.621567,8cb2c50921181ff,8cb2c50927283ff
113587,74381302,7438130201,349410.2813,6289669,349556.9063,6290333.5,10:30,10:35,2012-01-01 10:30:00,2012-01-01 10:35:00,-33.521961,-70.621567,-33.515990,-70.619877,8cb2c50927283ff,8cb2c50921181ff


Column names and column order are rearranged.

In [27]:
trips_cleaned = trips_cleaned.drop(['OrigenCoordX', 'OrigenCoordY', 'DestinoCoordX', 'DestinoCoordY','HoraIni', 'HoraFin', 'Viaje'], axis=1)
trips_cleaned = trips_cleaned.rename({'Persona': 'user_id'}, axis=1)


trips_to_file = trips_cleaned[['user_id', 'o_lon', 'o_lat', 'd_lon', 'd_lat', 'o_h3_cell', 'd_h3_cell', 'o_time', 'd_time']]
trips_to_file

,user_id,o_lon,o_lat,d_lon,d_lat,o_h3_cell,d_h3_cell,o_time,d_time
0,17343102,-70.774666,-33.531422,-70.735153,-33.495874,8cb2c540d0a89ff,8cb2c5429b60dff,2012-01-01 22:30:00,2012-01-01 23:40:00
1,17344101,-70.738205,-33.500007,-70.567232,-33.408776,8cb2c5429ae5dff,8cb2c519a994dff,2012-01-01 13:00:00,2012-01-01 14:45:00
2,17344101,-70.567232,-33.408776,-70.738205,-33.500007,8cb2c519a994dff,8cb2c5429ae5dff,2012-01-01 22:00:00,2012-01-01 23:30:00
3,17344103,-70.738205,-33.500007,-70.604904,-33.454153,8cb2c5429ae5dff,8cb2c554db113ff,2012-01-01 09:00:00,2012-01-01 09:55:00
4,17344103,-70.604904,-33.454153,-70.738205,-33.500007,8cb2c554db113ff,8cb2c5429ae5dff,2012-01-01 19:00:00,2012-01-01 21:30:00
...,...,...,...,...,...,...,...,...,...
113584,70679101,-70.737727,-33.429695,-70.736816,-33.430483,8cb2c55509217ff,8cb2c55509297ff,2012-01-01 12:40:00,2012-01-01 12:48:00
113585,74381301,-70.621567,-33.521961,-70.619877,-33.515990,8cb2c50927283ff,8cb2c50921181ff,2012-01-01 10:30:00,2012-01-01 10:45:00
113586,74381301,-70.619877,-33.515990,-70.621567,-33.521961,8cb2c50921181ff,8cb2c50927283ff,2012-01-01 12:00:00,2012-01-01 12:15:00
113587,74381302,-70.621567,-33.521961,-70.619877,-33.515990,8cb2c50927283ff,8cb2c50921181ff,2012-01-01 10:30:00,2012-01-01 10:35:00


In [28]:
trips_to_file.index.name="trip_id"
trips_to_file

,user_id,o_lon,o_lat,d_lon,d_lat,o_h3_cell,d_h3_cell,o_time,d_time
trip_id,,,,,,,,,
0,17343102,-70.774666,-33.531422,-70.735153,-33.495874,8cb2c540d0a89ff,8cb2c5429b60dff,2012-01-01 22:30:00,2012-01-01 23:40:00
1,17344101,-70.738205,-33.500007,-70.567232,-33.408776,8cb2c5429ae5dff,8cb2c519a994dff,2012-01-01 13:00:00,2012-01-01 14:45:00
2,17344101,-70.567232,-33.408776,-70.738205,-33.500007,8cb2c519a994dff,8cb2c5429ae5dff,2012-01-01 22:00:00,2012-01-01 23:30:00
3,17344103,-70.738205,-33.500007,-70.604904,-33.454153,8cb2c5429ae5dff,8cb2c554db113ff,2012-01-01 09:00:00,2012-01-01 09:55:00
4,17344103,-70.604904,-33.454153,-70.738205,-33.500007,8cb2c554db113ff,8cb2c5429ae5dff,2012-01-01 19:00:00,2012-01-01 21:30:00
...,...,...,...,...,...,...,...,...,...
113584,70679101,-70.737727,-33.429695,-70.736816,-33.430483,8cb2c55509217ff,8cb2c55509297ff,2012-01-01 12:40:00,2012-01-01 12:48:00
113585,74381301,-70.621567,-33.521961,-70.619877,-33.515990,8cb2c50927283ff,8cb2c50921181ff,2012-01-01 10:30:00,2012-01-01 10:45:00
113586,74381301,-70.619877,-33.515990,-70.621567,-33.521961,8cb2c50921181ff,8cb2c50927283ff,2012-01-01 12:00:00,2012-01-01 12:15:00


Save as a parquet file

In [29]:
trips_to_file.to_parquet('../data/unified_format/test_eod_unitrip.parquet', engine='pyarrow', compression='snappy')

## Code to aggregate trips and create flows in unitrip format (temp in notebook)


### Aggregate trips using h3 cells, with a specific level of resolution and filtering with a minimum number of trips per flow


Load the parquet to a dataframe with the important series

In [30]:
santiago_unitrip = pd.read_parquet('../data/unified_format/test_eod_unitrip.parquet', columns=['user_id', 'o_h3_cell', 'd_h3_cell'])
santiago_unitrip

,user_id,o_h3_cell,d_h3_cell
trip_id,,,
0,17343102,8cb2c540d0a89ff,8cb2c5429b60dff
1,17344101,8cb2c5429ae5dff,8cb2c519a994dff
2,17344101,8cb2c519a994dff,8cb2c5429ae5dff
3,17344103,8cb2c5429ae5dff,8cb2c554db113ff
4,17344103,8cb2c554db113ff,8cb2c5429ae5dff
...,...,...,...
113584,70679101,8cb2c55509217ff,8cb2c55509297ff
113585,74381301,8cb2c50927283ff,8cb2c50921181ff
113586,74381301,8cb2c50921181ff,8cb2c50927283ff


In [31]:
import h3 

flow_res = 7
santiago_unitrip['o_h3_cell'] = santiago_unitrip['o_h3_cell'].apply(lambda cell: h3.cell_to_parent(cell, flow_res))
santiago_unitrip['d_h3_cell'] = santiago_unitrip['d_h3_cell'].apply(lambda cell: h3.cell_to_parent(cell, flow_res))
santiago_unitrip

,user_id,o_h3_cell,d_h3_cell
trip_id,,,
0,17343102,87b2c540dffffff,87b2c5429ffffff
1,17344101,87b2c5429ffffff,87b2c519affffff
2,17344101,87b2c519affffff,87b2c5429ffffff
3,17344103,87b2c5429ffffff,87b2c554dffffff
4,17344103,87b2c554dffffff,87b2c5429ffffff
...,...,...,...
113584,70679101,87b2c5550ffffff,87b2c5550ffffff
113585,74381301,87b2c5092ffffff,87b2c5092ffffff
113586,74381301,87b2c5092ffffff,87b2c5092ffffff


In [32]:
filtered_unitrip = (santiago_unitrip
                    .pipe(lambda x: x[x.o_h3_cell != x.d_h3_cell])
                    .groupby(['user_id', 'o_h3_cell', 'd_h3_cell'])
                    .size()
                    )
filtered_unitrip.name = 'n_trips'
filtered_unitrip = filtered_unitrip.reset_index()
filtered_unitrip.index.name="trip_id"
filtered_unitrip

,user_id,o_h3_cell,d_h3_cell,n_trips
trip_id,,,,
0,10002001,87b2c5791ffffff,87b2c579cffffff,1
1,10002001,87b2c579cffffff,87b2c579effffff,1
2,10002002,87b2c5791ffffff,87b2c579effffff,1
3,10002003,87b2c0b70ffffff,87b2c579effffff,1
4,10002003,87b2c5791ffffff,87b2c0b70ffffff,1
...,...,...,...,...
72702,39994101,87b2c54f1ffffff,87b2c54f0ffffff,1
72703,70041001,87b2c5791ffffff,87b2c579cffffff,1
72704,70041001,87b2c579cffffff,87b2c5791ffffff,1


In [33]:
minimun_trips = 8
flows = (filtered_unitrip
         [filtered_unitrip.o_h3_cell != filtered_unitrip.d_h3_cell]
         .groupby(['o_h3_cell', 'd_h3_cell'])
         ['n_trips']
         .sum()
         .reset_index()
         .pipe(lambda x: x[x.n_trips > minimun_trips])
         .rename({'n_trips': 'count'}, axis=1)
        )
flows

,o_h3_cell,d_h3_cell,count
43,87b2c0853ffffff,87b2c085effffff,84
47,87b2c0853ffffff,87b2c08ecffffff,10
48,87b2c0853ffffff,87b2c08edffffff,61
77,87b2c085cffffff,87b2c085effffff,10
88,87b2c085effffff,87b2c0853ffffff,83
...,...,...,...
16333,87b2c5cdbffffff,87b2c5545ffffff,13
16345,87b2c5cdbffffff,87b2c5566ffffff,18
16346,87b2c5cdbffffff,87b2c5568ffffff,12
16362,87b2c5cdbffffff,87b2c5cd9ffffff,9


In [34]:
flows.to_parquet('../data/unified_format/test_eod_uniflow.parquet', engine='pyarrow', compression='snappy')

## Code to convert trips and flows (in unitrip format) to flowmap input (temp in notebook)


In [35]:
santiago_uniflow = pd.read_parquet('../data/unified_format/test_eod_uniflow.parquet')
santiago_uniflow

,o_h3_cell,d_h3_cell,count
43,87b2c0853ffffff,87b2c085effffff,84
47,87b2c0853ffffff,87b2c08ecffffff,10
48,87b2c0853ffffff,87b2c08edffffff,61
77,87b2c085cffffff,87b2c085effffff,10
88,87b2c085effffff,87b2c0853ffffff,83
...,...,...,...
16333,87b2c5cdbffffff,87b2c5545ffffff,13
16345,87b2c5cdbffffff,87b2c5566ffffff,18
16346,87b2c5cdbffffff,87b2c5568ffffff,12
16362,87b2c5cdbffffff,87b2c5cd9ffffff,9


In [36]:
unique_cells = pd.concat([santiago_uniflow['o_h3_cell'], santiago_uniflow['d_h3_cell']]).unique()
cells_to_id = {cell : i for i, cell in enumerate(unique_cells)}

In [37]:
santiago_uniflow['o_h3_cell'] = santiago_uniflow['o_h3_cell'].apply(lambda cell: cells_to_id[cell])
santiago_uniflow['d_h3_cell'] = santiago_uniflow['d_h3_cell'].apply(lambda cell: cells_to_id[cell])
santiago_uniflow

,o_h3_cell,d_h3_cell,count
43,0,2,84
47,0,3,10
48,0,4,61
77,1,2,10
88,2,0,83
...,...,...,...
16333,201,130,13
16345,201,158,18
16346,201,159,12
16362,201,199,9


In [38]:
santiago_uniflow_file = santiago_uniflow.rename({'o_h3_cell':'origin', 'd_h3_cell':'dest'}, axis=1)
santiago_uniflow_file.to_csv('../data/use_cases/flowmap/test_eod_flows.csv', index=False)

In [39]:
uniflow_location = pd.DataFrame({'name': cells_to_id.values(), 
                                 'lat':[h3.cell_to_latlng(cell)[0] for cell in cells_to_id.keys()],
                                 'lon':[h3.cell_to_latlng(cell)[1] for cell in cells_to_id.keys()]})
uniflow_location.index.name = 'id'
uniflow_location.to_csv('../data/use_cases/flowmap/test_eod_locations.csv')